In [5]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [6]:


# Set the paths for the dataset
train_directory = r'chest_xray\train'
test_directory = r'chest_xray\test'



In [7]:
# Image Data Generator with rescaling and basic augmentation
training_data_generator = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    shear_range=0.2,  # Random shear
    zoom_range=0.2,   # Random zoom
    horizontal_flip=True,  # Random horizontal flip
)

test_data_generator = ImageDataGenerator(rescale=1./255)

# Create the training and validation generators
train_generator = training_data_generator.flow_from_directory(
    train_directory,
    target_size=(150, 150),  # Resize images
    batch_size=32,           # Batch size
    class_mode='binary'      # Binary classification: normal vs pneumonia
)

test_generator = test_data_generator.flow_from_directory(
    test_directory,
    target_size=(150, 150),  # Resize images
    batch_size=32,
    class_mode='binary'
)

# Build the CNN pneumonia_detector_model
pneumonia_detector_model = Sequential()

# 1st Convolutional Layer
pneumonia_detector_model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation='relu'))
pneumonia_detector_model.add(MaxPooling2D(pool_size=(2, 2)))

# 2nd Convolutional Layer
pneumonia_detector_model.add(Conv2D(64, (3, 3), activation='relu'))
pneumonia_detector_model.add(MaxPooling2D(pool_size=(2, 2)))

# 3rd Convolutional Layer
pneumonia_detector_model.add(Conv2D(128, (3, 3), activation='relu'))
pneumonia_detector_model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output
pneumonia_detector_model.add(Flatten())

# Fully connected layer
pneumonia_detector_model.add(Dense(128, activation='relu'))
pneumonia_detector_model.add(Dropout(0.5))  # Dropout for regularization
pneumonia_detector_model.add(Dense(1, activation='sigmoid'))  # Binary output: 1 neuron with sigmoid

# Compile the pneumonia_detector_model
pneumonia_detector_model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',  # Binary classification
              metrics=['accuracy'])


Found 5232 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:

# Train the pneumonia_detector_model
pneumonia_detector_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Number of epochs to train
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)

# Evaluate the pneumonia_detector_model on the test set
test_loss, test_acc = pneumonia_detector_model.evaluate(test_generator)
print(f'Test accuracy: {test_acc:.2f}')


Epoch 1/10


C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 116s 661ms/step - accuracy: 0.7146 - loss: 0.6520 - val_accuracy: 0.8273 - val_loss: 0.4553
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - accuracy: 0.7812 - loss: 0.6228 - val_accuracy: 0.8125 - val_loss: 1.0946
Epoch 3/10


c:\Program Files\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


163/163 ━━━━━━━━━━━━━━━━━━━━ 110s 643ms/step - accuracy: 0.8793 - loss: 0.3001 - val_accuracy: 0.8405 - val_loss: 0.4228
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9062 - loss: 0.2319 - val_accuracy: 0.6250 - val_loss: 1.2388
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 105s 627ms/step - accuracy: 0.8979 - loss: 0.2545 - val_accuracy: 0.8586 - val_loss: 0.3348
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - accuracy: 0.9062 - loss: 0.2825 - val_accuracy: 0.7500 - val_loss: 0.5767
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 104s 617ms/step - accuracy: 0.8989 - loss: 0.2509 - val_accuracy: 0.8602 - val_loss: 0.3930
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 305us/step - accuracy: 0.9062 - loss: 0.1775 - val_accuracy: 0.9375 - val_loss: 0.1935
Epoch 9/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 108s 642ms/step - accuracy: 0.9198 - loss: 0.2272 - val_accuracy: 0.9079 - val_loss: 0.2227
Epoch 10/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 554us/step - accuracy: 0.9375 - loss: 0.2464 - val_

In [9]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Path to the image you want to test
image_path = r'E:\placement ki tayyari\x ray\archive\images.jpeg'

# Load the image and resize it to the same size used during training (150x150)
img = image.load_img(image_path, target_size=(150, 150))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Rescale pixel values (same as during training)
img_array = img_array / 255.0

# Add an extra dimension to match the input shape of the pneumonia_detector_model (batch_size, height, width, channels)
img_array = np.expand_dims(img_array, axis=0)

# Make the prediction
prediction = pneumonia_detector_model.predict(img_array)

# Get the predicted class (0 for 'normal', 1 for 'pneumonia')
if prediction[0][0] > 0.5:
    print("The X-ray shows signs of pneumonia.")
else:
    print("The X-ray appears to be normal.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
The X-ray shows signs of pneumonia.
